<a href="https://colab.research.google.com/github/amitkag85/AILearning/blob/master/DIY_Tool_Use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import inspect
import json
from typing import Any, Callable, Dict, List

In [ ]:
def get_schema(item: Callable) -> Dict[str, Dict[str, Any]]:
    """Converts a function into a dict describing what it does."""
    schema = {
        "name": item.__name__,
        "description": str(inspect.getdoc(item)),
        "signature": str(inspect.signature(item)),
        "output": str(inspect.signature(item).return_annotation),
    }
    return {item.__name__: schema}

In [ ]:
def multiplyer(a: int, b: int) -> int:
    """
    Multiplies two numbers together.

    :param a: The first number to multiply.
    :type a: int

    :param b: The second number to multiply.
    :type b: int

    :return: The product of the two numbers.
    """
    return a * b

# print(multiplyer.__name__)
# print(inspect.getdoc(multiplyer))
# print(inspect.signature(multiplyer))
# print(inspect.signature(multiplyer).return_annotation)

print(json.dumps(get_schema(multiplyer), indent=4))

{
    "multiplyer": {
        "name": "multiplyer",
        "description": "Multiplies two numbers together.\n\n:param a: The first number to multiply.\n:type a: int\n\n:param b: The second number to multiply.\n:type b: int\n\n:return: The product of the two numbers.",
        "signature": "(a: int, b: int) -> int",
        "output": "<class 'int'>"
    }
}


In [ ]:
a = {"a": 1}
b = {"b": 2}

a | b

# Prompting for Tool Use

This class is a wrapper around a few-shot prompt for tool use. It is passed a prompt and a list of python funtions. The functions are then converted to JSON descriptions via `get_schema` and passed into the prompt template along with the user input.

In [ ]:
class MyMethodCaller:
    """Wrapper for constructing the function calling prompt."""
    def __init__(self):
        pass

    def get_schema(self, item: Callable) -> Dict[str, Dict[str, Any]]:
        """Converts a function into a dict describing what it does."""
        schema = {
            "name": item.__name__,
            "description": str(inspect.getdoc(item)),
            "signature": str(inspect.signature(item)),
            "output": str(inspect.signature(item).return_annotation),
        }
        return {item.__name__: schema}

    def get_schemas(self, functions: List[Callable]) -> Dict[str, Dict[str, Any]]:
        """Converts a list of functions into a dict of descriptions for what they do."""
        out = {}
        for f in functions:
            out = out | self.get_schema(f)
        return out

    def run(self, query: str, functions: List[Callable]):
        # convert the provided functions into their JSON descriptions
        schema = self.get_schemas(functions)
        # create a dict of each function name to its function for later
        function_dict = {f.__name__: f for f in functions}

        # construct the prompt as a one-shot example that details the schema and how to interpret it
        prompt = f"""
You are a helpful assistant designed to output JSON.
You are bad at math, but good at using a variety of functions..
Given the following function schema
<< {schema} >>
and query
<< {query} >>
extract the parameters values from the query, in a valid JSON format.
Example:
Input:
query: "How is the weather in Hawaii and London right now in International units?"
schema:
{{
    "get_weather": {{
        "name": "get_weather",
        "description": "Useful to get the weather in a specific location",
        "signature": "(location: str, degree: str) -> str",
        "output": "<class 'str'>",
    }}
}}

Result: [{{
    "function": "get_weather",
    "inputs": {{
        "location": "Hawaii",
        "degree": "Celsius",
    }}
}},
{{
    "function": "get_weather",
    "inputs": {{
        "location": "London",
        "degree": "Celsius",
    }}
}}]

output only the JSON. if there are no functions to call, output
an empty list.

Input:
query: {query}
schema: {schema}
Result:
        """

        return prompt


Here are some functions that we can pass to the function caller. They are regular python functions, but should contain:

* type annotations
* docstrings including descriptions of all arguments and their types

In [ ]:
# Function to be called described using the Python docstring format
def add_two_numbers(first_number: int, second_number: int) -> int:
    """
    Adds two numbers together.

    :param first_number: The first number to add.
    :type first_number: int

    :param second_number: The second number to add.
    :type second_number: int

    :return: The sum of the two numbers.
    """
    return first_number + second_number


def multiply_two_numbers(first_number: int, second_number: int) -> int:
    """
    Multiplies two numbers together.

    :param first_number: The first number to multiply.
    :type first_number: int

    :param second_number: The second number to multiply.
    :type second_number: int

    :return: The product of the two numbers.
    """
    return first_number * second_number


def say_hello(name: str) -> str:
    """
    Creates a greeting.

    :param name: The name of the person.
    :type name: str

    :return: The formatted greeting.
    """
    return f'Hello {name}! How are you today?'

In [ ]:
add_two_numbers(**{'first_number':1, 'second_number':2})

Instantiate our class and define a list of available functions.

In [ ]:
fun = MyMethodCaller()
functions = [add_two_numbers, say_hello, multiply_two_numbers]

This function will take a list of function call JSON objects and collect the results.

In [ ]:
def collect_results(function_inputs):
    function_dict = {f.__name__: f for f in functions}
    out = []
    for x in function_inputs:
        try:
            f = function_dict.get(x['function'])
            out.append(f(**x['inputs']))
        except KeyError as e:
            print(e)
            print(f)
            print(x)
        except TypeError as e:
            print(e)
            print(f)
            print(x)

    return out

In [ ]:
result = fun.run("Add 2 and five. Also what is 1 + 2?", functions)
# result = fun.run("1 + 2 * 3", functions)
print(result)


You are a helpful assistant designed to output JSON.
You are bad at math, but good at using a variety of functions..
Given the following function schema
<< {'add_two_numbers': {'name': 'add_two_numbers', 'description': 'Adds two numbers together.\n\n:param first_number: The first number to add.\n:type first_number: int\n\n:param second_number: The second number to add.\n:type second_number: int\n\n:return: The sum of the two numbers.', 'signature': '(first_number: int, second_number: int) -> int', 'output': "<class 'int'>"}, 'say_hello': {'name': 'say_hello', 'description': 'Creates a greeting.\n\n:param name: The name of the person.\n:type name: str\n\n:return: The formatted greeting.', 'signature': '(name: str) -> str', 'output': "<class 'str'>"}, 'multiply_two_numbers': {'name': 'multiply_two_numbers', 'description': 'Multiplies two numbers together.\n\n:param first_number: The first number to multiply.\n:type first_number: int\n\n:param second_number: The second number to multiply

In [ ]:
result = fun.run("My name is Bob, nice to meet you.", functions)
print(result)


You are a helpful assistant designed to output JSON.
You are bad at math, but good at using a variety of functions..
Given the following function schema
<< {'add_two_numbers': {'name': 'add_two_numbers', 'description': 'Adds two numbers together.\n\n:param first_number: The first number to add.\n:type first_number: int\n\n:param second_number: The second number to add.\n:type second_number: int\n\n:return: The sum of the two numbers.', 'signature': '(first_number: int, second_number: int) -> int', 'output': "<class 'int'>"}, 'say_hello': {'name': 'say_hello', 'description': 'Creates a greeting.\n\n:param name: The name of the person.\n:type name: str\n\n:return: The formatted greeting.', 'signature': '(name: str) -> str', 'output': "<class 'str'>"}, 'multiply_two_numbers': {'name': 'multiply_two_numbers', 'description': 'Multiplies two numbers together.\n\n:param first_number: The first number to multiply.\n:type first_number: int\n\n:param second_number: The second number to multiply

In [ ]:
result = fun.run("What's for lunch?", functions)
print(result)

In [ ]:
inpt = json.loads('''
[
{
"function": "add_two_numbers",
"inputs": {
"first_number": 2,
"second_number": 5
}
},
{
"function": "add_two_numbers",
"inputs": {
"first_number": 1,
"second_number": 2
}
}
]''')

print(collect_results(inpt))

In [ ]:
inpt = json.loads('''
[{"function": "say_hello", "inputs": {"name": "Bob"}}]''')

print(collect_results(inpt))

In [ ]:
query = "1 + 2 * 3"
call = "multiply_two_numbers(2, 3)"
res = 6 # eval(call)
prompt = """please rewirte the query, replacing the part referenced by `{call}` with {res}.
please only provide the rewritten query.

query: "{query}"
"""
print(prompt.format(call=call, res=res, query=query))
